In [ ]:
import torch
from PIL import Image
import numpy as np
import cv2

from invokeai.backend.vto_workflow.overlay_pattern import generate_dress_mask, multiply_images
from invokeai.backend.vto_workflow.extract_channel import extract_channel, ImageChannel
from invokeai.backend.vto_workflow.seamless_mapping import map_seamless_tiles




In [ ]:
 # Load the model image.
model_image = Image.open("/home/ryan/src/InvokeAI/invokeai/backend/vto_workflow/dress.jpeg")

# Load the pattern image.
pattern_image = Image.open("/home/ryan/src/InvokeAI/invokeai/backend/vto_workflow/pattern1.jpg")

In [ ]:
mask = generate_dress_mask(model_image)

In [ ]:
# Visualize mask
model_image_np = np.array(model_image)
masked_model_image = (model_image_np * np.expand_dims(mask, -1).astype(np.float32)).astype(np.uint8)
mask_image = Image.fromarray(masked_model_image)
mask_image

In [ ]:
shadows = extract_channel(np.array(model_image), ImageChannel.LAB_L)

In [ ]:
# Visualize masked shadows
masked_shadows = (shadows * mask).astype(np.uint8)
masked_shadows_image = Image.fromarray(masked_shadows)
masked_shadows_image

In [ ]:
# Tile the pattern.
expanded_pattern = map_seamless_tiles(seamless_tile=pattern_image, target_hw=(model_image.height, model_image.width), num_repeats_h=10.0)


In [ ]:
# Multiply the pattern by the shadows.
pattern_with_shadows = multiply_images(expanded_pattern, shadows)
pattern_with_shadows

In [ ]:
# Merge the pattern with the model image.
pattern_with_shadows_np = np.array(pattern_with_shadows)
merged_image = np.where(mask[:, :, None],  pattern_with_shadows_np,model_image_np)
merged_image = Image.fromarray(merged_image)
merged_image